# str

In [ ]:
#| default_exp repr_str

In [ ]:
# |hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
#| hide
#| export
from typing import Optional, Union
import torch

In [ ]:
# |exporti
class __PrinterOptions(object):
    precision: int = 3
    threshold_max: int = 3 # .abs() larger than 1e3 -> Sci mode
    threshold_min: int = -4 # .abs() smaller that 1e-4 -> Sci mode
    sci_mode: Optional[bool] = None # None = auto. Otherwise, force sci mode.
    indent: int = 2 # Indent for .deeper()
    color: bool = True


In [ ]:
# |export
PRINT_OPTS = __PrinterOptions()

In [ ]:
# |hide
# |exporti
# Do we want this float in decimal or scientific mode?
def sci_mode(f: float):
    return (abs(f) < 10**(PRINT_OPTS.threshold_min) or
            abs(f) > 10**PRINT_OPTS.threshold_max)

In [ ]:
# |hide
test_eq(sci_mode(1.), False)
test_eq(sci_mode(0.00001), True)
test_eq(sci_mode(10000000), True)

# It would be fine either way, both `e` and `f` formats handle those.
test_eq(sci_mode(float('nan')), False)
test_eq(sci_mode(float('inf')), True) 

In [ ]:
# |hide

# What's happening in the cell below
fmt = f"{{:.{4}{'e'}}}"
fmt, fmt.format(1.23)

('{:.4e}', '1.2300e+00')

In [ ]:
# |export

# Convert a tensor or scalar into a string.
# This only looks good for small tensors, which is how it's intended to be used.
def pretty_str(t: Union[torch.Tensor, float, int]):
    """A slightly better way to print `floaty` values"""

    if isinstance(t, int):
        return '{}'.format(t)
    elif isinstance(t, float):
        if t == 0.:
            return "0."

        sci = (PRINT_OPTS.sci_mode or
                (PRINT_OPTS.sci_mode is None and sci_mode(t)))
        # The f-string will generate something like "{.4f}", which is used
        # to format the value.
        return f"{{:.{PRINT_OPTS.precision}{'e' if sci else 'f'}}}".format(t)
    elif t.dim() == 0:
            return pretty_str(t.item())
    else:
        slices = [pretty_str(t[i]) for i in range(0, t.size(0))]
        return '[' + ", ".join(slices) + ']'

In [ ]:
# |hide
torch.manual_seed(42)
randoms = torch.randn(100)

In [ ]:
nasties = randoms[:12].clone()

nasties[0] *= 10000
nasties[1] /= 10000
nasties[3] = float('inf')
nasties[4] = float('-inf')
nasties[5] = float('nan')
nasties = nasties.reshape((2,6))


In [ ]:
pretty_str(nasties)

'[[1.927e+04, 0.000, 0.901, inf, -inf, nan], [-0.043, -1.605, -0.752, 1.649, -0.392, -1.404]]'

In [ ]:
# |hide
test_eq(pretty_str(nasties), '[[1.927e+04, 0.000, 0.901, inf, -inf, nan], [-0.043, -1.605, -0.752, 1.649, -0.392, -1.404]]')

In [ ]:
# |exporti
# |hide
def space_join(lst):
    # Join non-empty list elements into a space-sepaeated string
    return " ".join( [ l for l in lst if l] )

In [ ]:
# |hide
test_eq(space_join(["Hello", None, "World"]), 'Hello World')

In [ ]:
#| exporti

class StrProxy():
    def __init__(self, t: torch.Tensor, plain=False, verbose=False, depth=0, lvl=0, color=None):
        self.t = t
        self.plain = plain
        self.verbose = verbose
        self.depth=depth
        self.lvl=lvl
        self.color=color

    @torch.no_grad()
    def to_str(self):
        t = self.t
        if self.plain or t.is_complex():
            return torch._tensor_str._tensor_str(t, indent=0)

        color = PRINT_OPTS.color if self.color is None else self.color
        
        grey_style = "\x1b[38;2;127;127;127m" if color else ""
        red_style = "\x1b[31m" if color else ""
        end_style = "\x1b[0m" if color else ""

        dtnames = { torch.float32: "",
                    torch.float16: "f16",
                    torch.float64: "f64",
                    torch.uint8: "u8",
                    torch.int32: "i32",
                }


        tname = "tensor" if type(t) is torch.Tensor else type(t).__name__
        dev = str(t.device) if t.device.type != "cpu" else None
        dtype = dtnames[t.dtype] if t.dtype in dtnames else str(t.dtype)[6:]


        grad_fn = t.grad_fn.name() if t.grad_fn else None
        # All tensors along the compute path actually have required_grad=True.
        # Torch __repr__ just dones not show it.
        grad = "grad" if t.requires_grad else None

        shape = str(list(t.shape))

        # Later, we might be indexing 't' with a bool tensor derived from it. 
        # THis takes 4x memory and will result in a CUDA OOM if 't' is very large.
        # Move it to the cpu now - it won't matter for small tensors, and for
        # very large ones we trade a CUDA OOM for a few seconds delay.
        t = t.detach().cpu()

        zeros = grey_style+"all_zeros"+end_style if t.eq(0.).all() and t.numel() > 1 else None
        pinf = red_style+"+inf!"+end_style if t.isposinf().any() else None
        ninf = red_style+"-inf!"+end_style if t.isneginf().any() else None
        nan = red_style+"nan!"+end_style if t.isnan().any() else None


        attention = space_join([zeros,pinf,ninf,nan])

        x = ""
        summary = f"n={t.numel()}" if t.numel() > 5 else None
        if not zeros:
            if t.numel() <= 10: x = pretty_str(t)
            
            # Make sure it's float32. Also, we calculate stats on good values only.

            ft = t[ torch.isfinite(t) ].float()

            minmax = f"x∈[{pretty_str(ft.min())}, {pretty_str(ft.max())}]" if t.numel() > 2 and ft.numel() > 2 else None
            meanstd = f"μ={pretty_str(ft.mean())} σ={pretty_str(ft.std())}" if t.numel() >= 2 and ft.numel() >= 2 else None
            numel = f"n={t.numel()}" if t.numel() > 5 and max(t.shape) != t.numel() else None

            summary = space_join([numel, minmax, meanstd])




        res = tname + space_join([  shape,
                                    summary,
                                    dtype,
                                    grad,
                                    grad_fn,
                                    dev,
                                    attention,
                                    x if not self.verbose else None])

        if self.verbose:
            res += "\n" + torch._tensor_str._tensor_str(t, indent=0)

        if self.depth and t.dim() > 1:
            res += "\n" + "\n".join([
                " "*PRINT_OPTS.indent*(self.lvl+1) +
                str(StrProxy(t[i,:], depth=self.depth-1, lvl=self.lvl+1))
                for i in range(t.shape[0])])

        return res
    
    def __repr__(self):
        return self.to_str()

    def __call__(self, depth=0):
        return StrProxy(self.t, depth=depth)


Would be _lovely_ if you could see all the important tensor stats too?

In [ ]:
# |export
def lovely(t: torch.Tensor, verbose=False, plain=False, depth=0, color=None):
    return StrProxy(t, verbose=verbose, plain=plain, depth=depth, color=color)

In [ ]:
print(lovely(randoms[0]))
print(lovely(randoms[:2]))
print(lovely(randoms[:6].view(2, 3))) # More than 2 elements -> show statistics
print(lovely(randoms[:11])) # More than 10 -> suppress data output


tensor[] 1.927
tensor[2] μ=1.707 σ=0.311 [1.927, 1.487]
tensor[2, 3] n=6 x∈[-2.106, 1.927] μ=0.276 σ=1.594 [[1.927, 1.487, 0.901], [-2.106, 0.678, -1.235]]
tensor[11] x∈[-2.106, 1.927] μ=0.046 σ=1.384


In [ ]:
# |hide
test_eq(str(lovely(randoms[0])), "tensor[] 1.927")
test_eq(str(lovely(randoms[:2])), "tensor[2] μ=1.707 σ=0.311 [1.927, 1.487]")
test_eq(str(lovely(randoms[:6].view(2, 3))), "tensor[2, 3] n=6 x∈[-2.106, 1.927] μ=0.276 σ=1.594 [[1.927, 1.487, 0.901], [-2.106, 0.678, -1.235]]")
test_eq(str(lovely(randoms[:11])), "tensor[11] x∈[-2.106, 1.927] μ=0.046 σ=1.384")

In [ ]:
grad = torch.tensor(1., requires_grad=True)
print(lovely(grad)); print(lovely(grad+1))

tensor[] grad 1.000
tensor[] grad AddBackward0 2.000


In [ ]:
# |hide
test_eq(str(lovely(grad)), "tensor[] grad 1.000")
test_eq(str(lovely(grad+1)), "tensor[] grad AddBackward0 2.000")

In [ ]:
if torch.cuda.is_available():
    print(lovely(torch.tensor(1., device=torch.device("cuda:0"))))
    test_eq(str(lovely(torch.tensor(1., device=torch.device("cuda:0")))), "tensor[] cuda:0 1.000")

tensor[] cuda:0 1.000


Do we have __any__ floating point nasties? Is the tensor __all__ zeros?

In [ ]:
PRINT_OPTS.color=True

# Statistics and range are calculated on good values only, if there are at lest 3 of them.
lovely(nasties)

tensor[2, 6] n=12 x∈[-1.605, 1.927e+04] μ=2.141e+03 σ=6.423e+03 +inf! -inf! nan!

In [ ]:
lovely(nasties, color=False)

tensor[2, 6] n=12 x∈[-1.605, 1.927e+04] μ=2.141e+03 σ=6.423e+03 +inf! -inf! nan!

In [ ]:
lovely(torch.tensor([float("nan")]*11))

tensor[11] nan!

In [ ]:
lovely(torch.zeros(12))

tensor[12] n=12 all_zeros

In [ ]:
test_eq(str(lovely(nasties)),
    'tensor[2, 6] n=12 x∈[-1.605, 1.927e+04] μ=2.141e+03 σ=6.423e+03 \x1b[31m+inf!\x1b[0m \x1b[31m-inf!\x1b[0m \x1b[31mnan!\x1b[0m')
test_eq(str(lovely(torch.tensor([float("nan")]*11))), 'tensor[11] \x1b[31mnan!\x1b[0m')
test_eq(str(lovely(torch.zeros(12))), 'tensor[12] n=12 \x1b[38;2;127;127;127mall_zeros\x1b[0m')

In [ ]:
torch.set_printoptions(linewidth=120)
lovely(nasties, verbose=True)

tensor[2, 6] n=12 x∈[-1.605, 1.927e+04] μ=2.141e+03 σ=6.423e+03 +inf! -inf! nan!
[[ 1.9269e+04,  1.4873e-04,  9.0072e-01,         inf,        -inf,         nan],
 [-4.3067e-02, -1.6047e+00, -7.5214e-01,  1.6487e+00, -3.9248e-01, -1.4036e+00]]

In [ ]:
lovely(nasties, plain=True)

[[ 1.9269e+04,  1.4873e-04,  9.0072e-01,         inf,        -inf,         nan],
 [-4.3067e-02, -1.6047e+00, -7.5214e-01,  1.6487e+00, -3.9248e-01, -1.4036e+00]]

In [ ]:
image = torch.load("mysteryman.pt")
image[1,100,100] = float('nan')

lovely(image, depth=1)

tensor[3, 196, 196] n=115248 x∈[-2.118, 2.640] μ=-0.388 σ=1.073 nan!
  tensor[196, 196] n=38416 x∈[-2.118, 2.249] μ=-0.324 σ=1.036
  tensor[196, 196] n=38416 x∈[-1.966, 2.429] μ=-0.274 σ=0.973 nan!
  tensor[196, 196] n=38416 x∈[-1.804, 2.640] μ=-0.567 σ=1.178

In [ ]:
if torch.cuda.is_available():
    cudamem = torch.cuda.memory_allocated()
    print(f"before allocation: {torch.cuda.memory_allocated()=}")
    numbers = torch.randn((3, 1024, 1024), device="cuda") # 12Mb image
    torch.cuda.synchronize()

    print(f"after allocation: {torch.cuda.memory_allocated()=}")
    # Note, the return value of lovely() is not a string, but a
    # StrProxy that holds reference to 'numbers'. You have to del
    # any references to it.
    display(lovely(numbers) )
    print(f"after repr: {torch.cuda.memory_allocated()=}")
    
    del numbers
    # torch.cuda.memory.empty_cache()

    print(f"after cleanup: {torch.cuda.memory_allocated()=}")
    test_eq(cudamem >= torch.cuda.memory_allocated(), True)

before allocation: torch.cuda.memory_allocated()=0
after allocation: torch.cuda.memory_allocated()=12582912


tensor[3, 1024, 1024] n=3145728 x∈[-5.325, 5.150] μ=-0.000 σ=0.999 cuda:0

after repr: torch.cuda.memory_allocated()=12582912
after cleanup: torch.cuda.memory_allocated()=0


In [ ]:
# We don't really supposed complex numbers yet
c = torch.randn(10, dtype=torch.complex64)
c

tensor([ 0.5135+0.0645j, -0.2751+0.3733j, -0.2552-0.0428j,  0.0518+0.5789j,  1.0469+0.2439j, -1.0070-0.0823j,
         0.1538-0.0330j, -1.0137-0.4006j, -0.3007+0.1856j, -1.0176+0.3687j])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()